In [18]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
from sodapy import Socrata
from geopandas import gpd

In [2]:
client = Socrata("data.cityofchicago.org", 'EjrjYzG6YAkBx7bPBzME8jD4c')
results = client.get("w3hi-cfa4",limit = 500000)

complaints = pd.DataFrame.from_records(results)

In [41]:
split = complaints['beat'].str.split('|').apply(pd.Series, 1).stack()
split.index = split.index.droplevel(-1)
split.name = 'beat'
del complaints['beat']
split = split.apply(lambda x: x.strip())
complaints = complaints.join(split, how ="right")
complaints['complaint_date'] =  pd.to_datetime(complaints['complaint_date'], format='%Y%m%dT%H:%M:%S.%f')
complaints['complaint_year'] = complaints.complaint_date.dt.year

In [38]:
beats_gpd = gpd.read_file('policebeats.shp')

In [44]:
complaints['complaint_year'].value_counts()

2010    4291
2009    3937
2011    3740
2008    3443
2012    3230
2013    2424
2014    2082
2015    1629
2016    1392
2017    1251
2007    1093
2018     128
Name: complaint_year, dtype: int64